In [2]:
import os 
from os import path 
import shutil
import csv
from PIL import Image
import numpy as np
import tabulate 

def create_directory(path,directory_name):

    directory_dest = path + "/" + directory_name
    os.mkdir(directory_dest)
    
def create_dictionary_files(list_files,resource):
    dictionary_files={}
    for name in list_files:
        if name == "recap.csv":
            continue
        
        resource_file = resource + "/" + name
        if path.isfile(resource_file) :
            size = path.getsize(resource_file)
            name_split = path.splitext(name)
            # If we were to consider more different types of extension 
            # it would be enough to create three sets (images,docs,audio) of type set() in which you would need to 
            # initialize all the extensions we want to consider for each file type.
            if name_split[1] == ".jpg" or name_split[1] == ".png" or name_split[1] == ".jpeg":
                dictionary_files[name]= {"type" : "images","size":size}
            elif name_split[1] == ".txt" or name_split[1] == ".odt" or name_split[1] == ".docx":
                dictionary_files[name]= {"type" : "docs","size":size}
            elif name_split[1] == ".mp3":
                dictionary_files[name]= {"type" : "audio","size":size}
            else: 
                dictionary_files[name]= {"type" : "altro","size":size}

    return dictionary_files

def create_set_types(path_files):
    exist = False
    set_types = set()
    name_elements = os.listdir(path_files)
    name_files = list(name_elements)
    for name in name_elements: 
        if path.isdir(path_files + "/" + name):
            name_files.remove(name)
            set_types.add(name)
        elif name == "recap.csv":
            exist = True 

    return set_types,name_files,exist

def iteration_files(position_files):
    # To check if the directory already exists, I create a set with the three categories (images, documents, audio).
    set_types,name_files, exist = create_set_types(position_files)
    # I create a dictionary to match each file to its type (image, document, audio).
    dictionary_files = create_dictionary_files(name_files,position_files)

    if dictionary_files == None :
        exit("There are no free files on the macro directory")
    #Iteration in the file
    for name,info in dictionary_files.items():
        print(f"{name} type:{info['type']} size:{info['size']}\n")
        # Check if the folder already exists
        # If it does not exist, add the directory and move the files to the directory.
        if info["type"] not in set_types:
            set_types.add(info["type"])
            create_directory(position_files,info["type"])

            # move files to the right directory 
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)

        else:
            # move files to the right directory 
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)
    
    return dictionary_files, exist

def recap(path_position,dictionary,exist):

    name_path = path_position + "/" + "recap.csv"
    if not exist :
        with open(name_path,'w', encoding="UTF8", newline="") as f:
            writer = csv.writer(f)
            # Header
            writer.writerow(["name","type","size(B)"])
    
    with open(name_path,"a", encoding="UTF8", newline="") as f:
        writer = csv.writer(f)
        for name,info in dictionary.items():
            name_clean = path.splitext(name)[0]
            #Files  
            writer.writerow([name_clean, info["type"], str(info["size"])])

def create_array_informations(name,directory):

    resource = directory + "/" + name
    im = Image.open(resource)
    a = np.array(im)
    return a

def create_list_info(list,direct_images):
    # I chose to use a list because it was better to print with the form "tabulate". 
    list_info = []
    # Iteration in the image directory
    for name in list:
        name_clean = os.path.splitext(name)[0]
        x = create_array_informations(name,direct_images)
        tipology = np.shape(x)

        # Find the mean values (grayscale, R,G,B,ALPHA) for each image.
        if len(tipology) == 2:
            grayscale = round(np.mean(x),2)
            # I insert within the table the values we are interested in. 
            list_info.append([name_clean,tipology[0],tipology[1],grayscale,0.00,0.00,0.00,0.00])
        else:
            alpha = 0 
            tupla_averages = np.mean(x,axis=(0,1))
            if tipology[2] == 4 : 
                alpha = round(tupla_averages[3],2)
            R = round(tupla_averages[0],2)
            G = round(tupla_averages[1],2)
            B = round(tupla_averages[2],2)
            # I insert within the table the values we are interested in.
            list_info.append([name_clean,tipology[0],tipology[1],0.00,R,G,B,alpha])
      
    return list_info

def printa_tabule(info):
    headers = ["name","height","width","greyscale","R","G","B","ALPHA"]
    print(tabulate.tabulate(info,headers,tablefmt="grid"))

def main():
     
    position_files= "" ## INSERT Path of the Location of the directory "file" 
    informations_of_files, exist = iteration_files(position_files)
    recap(position_files,informations_of_files, exist)

    directory_images = position_files + "/images"
    list_of_images = os.listdir(directory_images)
    # Creating a database via table with the information we need for each image. 
    list_info = create_list_info(list_of_images,directory_images)
    # Table print 
    printa_tabule( list_info ) 

if __name__ == "__main__":
    main()


7.Cash_Budgets.xlsx type:altro size:30506

CVP handout.docx type:docs size:1661357

Cyber Security Assignment 1 (1).pptx type:altro size:10514266

Cyber Security Assignment 1.pptx type:altro size:10514266

Document1 (1).docx type:docs size:18030

Document1 (2).docx type:docs size:17195

Ocean Lotus.pptx type:altro size:389407

OilRig.pdf type:altro size:369722

Online_Teaching_Platforms.docx type:docs size:142987

+---------------------------------------+----------+---------+-------------+--------+--------+--------+---------+
| name                                  |   height |   width |   greyscale |      R |      G |      B |   ALPHA |
+=======================================+==========+=========+=============+========+========+========+=========+
| bw                                    |      512 |     512 |       21.48 |   0    |   0    |   0    |    0    |
+---------------------------------------+----------+---------+-------------+--------+--------+--------+---------+
| ciao      